# Zero-shot classification with transformer-based pre-trained models

This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with the Splunk Machine Learning Toolkit (MLTK) Container for TensorFlow. This script contains an example of how to conduct **End-to-End zero-shot classification task over English text (eg. dialogue transcript summaries)** using the transformers pipeline library.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [26]:
# this definition exposes all python module imports that should be available in all subsequent commands
# import sys
# sys.path.insert(1, '/opt/conda/lib/python3.8/site-packages')

import json
import numpy as np
import pandas as pd
import transformers
from transformers import pipeline

import os
path = os.path.abspath(np.__file__)
print("numpy path in runtime: " + path)
print("numpy version in runtime: " + np.__version__)

# tokenizer = AutoTokenizer.from_pretrained("/opt/conda/lib/python3.8/site-packages/samsum_tok", local_files_only=True)
# summarizer = AutoModelForSeq2SeqLM.from_pretrained("/opt/conda/lib/python3.8/site-packages/samsum", local_files_only=True)

# tokenizer = AutoTokenizer.from_pretrained("philschmid/bart-large-cnn-samsum")
# # tokenizer.save_pretrained("/opt/conda/lib/python3.8/site-packages/samsum_tok")
# model = AutoModelForSeq2SeqLM.from_pretrained("philschmid/bart-large-cnn-samsum")
# model.save_pretrained("/opt/conda/lib/python3.8/site-packages/samsum")

MODEL_DIRECTORY = "/srv/app/model/data/"

numpy path in runtime: /opt/conda/lib/python3.8/site-packages/numpy/__init__.py
numpy version in runtime: 1.22.1


In [27]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.22.1
pandas version: 1.4.3


In [3]:
# from transformers import pipeline
# summarizer = pipeline("summarization", model="lidiya/bart-base-samsum")

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| makeresults
| eval text = "Client has a problem with the product he bought from the company this week. The size of the product in the picture is bigger than the size that is written in the description. Customer Service Representative will send him an accurate description of the method of use and he will follow up with one of the customer service representatives in the department of inquiries to clarify all the details."
| fit MLTKContainer algo=transformers_classification_en mode=stage epochs=100 text into app:transformers_classification_model as text

In [28]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [29]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("transformers_classification_model")
print(df)
print(df.shape)
print(str(param))

                                                text
0  Customer Service Representative from the KDDI ...
(1, 1)
{'options': {'params': {'algo': 'transformers_classification_en2', 'mode': 'stage', 'epochs': '100'}, 'args': ['text'], 'feature_variables': ['text'], 'model_name': 'transformers_classification_model', 'output_name': 'text', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '600', 'max_inputs': '100000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '30', 'max_score_time': '600', 'use_sampling': 'true'}, 'kfold_cv': None}, 'feature_variables': ['text']}


## Stage 2 - create and initialize a model

In [30]:
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    
    print("debug: start model initialization")
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    model = classifier
    print("debug: model initialized")
    return model

In [31]:
model = init(df,param)

debug: start model initialization
debug: model initialized


## Stage 3 - fit the model

Note that for this algorithm the model is pre-trained and therefore this stage is a placeholder only

In [32]:
# returns a fit info json object
def fit(model,df,param):
    returns = {}
    return returns

## Stage 4 - apply the model

The variable **threshold** is used to adjust the posterior probability threshold for a tag to be returned when called. 
For example, threshold=0.1 returns the tags with scores higher than 0.1 from the pre-defined classes. The variable **ret_num** denotes the maximum returned tag number.

In [44]:
def apply(model,df,param):
    threshold = 0.1
    ret_num = 6
    # define tags based on user-cases
    classes = [
        'technical support',
        'telephone malfunction',
        'television malfunction',
        'wifi malfunction',
        'line disturbance',
        'payment issue',
        'payment method',
        'refund inquiry'
        'product return',
        'product exchange',
        'purchase cancellation',
        'product questions',
        'user registration',
        'password reset',
        'user cancellation',
        'user information change',
        'follow up inquiries',
        'other',
        'spam'
    ]
    
    X = df[param['feature_variables']].values.tolist()
    temp_data=list()
    
    print("debug: read input")

    for i in range(len(X)):
        results = model(str(X[i]), classes)
        vals = results["scores"]
        labels = results["labels"]
#         print(vals)
        ret = ""
        for i in range(ret_num):
            if vals[i] > threshold:
                ret += labels[i]
                ret += ", "
        if ret == "":
            ret += labels[0]
        temp_data.append(ret)
        
    column_names=["tagging_result"]
    returns=pd.DataFrame(temp_data, columns=column_names)
    print("debug: finished")
        
    return returns

In [45]:
returns = apply(model,df,param)
print(returns)

debug: read input
debug: finished
                                 tagging_result
0  wifi malfunction, technical support, other, 


## Stage 5 - save the model

In [35]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # model will not be saved or reloaded as it is pre-built
    return model

## Stage 6 - load the model

In [36]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    # model will not be saved or reloaded as it is pre-built
    return model

## Stage 7 - provide a summary of the model

In [37]:
# return model summary
def summary(model=None):
    returns = {}
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code